In [1]:
import numpy as np 
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt

In [132]:
def shift_array(x, delta):
    return np.roll(x, delta)

def compute_Y(row, sigma):
    # Estraiamo i pesi
    w_noise, w_stim = row['(noise_weight , signal_weight)']
    
    # Estraiamo il ritardo
    delta = row['delay']
    
    # Recuperiamo le serie di X
    Xn = row['X_noise']      # Rumore
    Xs = row['X_stimulus']   # Segnale
    
    # Shiftiamo le serie di delta
    Xn_shifted = shift_array(Xn, delta)
    Xs_shifted = shift_array(Xs, delta)
    
    # Generiamo il rumore gaussiano
    noise = np.random.normal(0, sigma, len(Xn))
    
    # Calcoliamo Y(t)
    Y = w_stim * Xs_shifted + w_noise * Xn_shifted + noise
    
    return Y


def plot_Y_timeseries(dataset, row_indices):
    
    # Se l'utente passa un singolo intero, trasformiamolo in lista per uniformità
    if isinstance(row_indices, int):
        row_indices = [row_indices]
        
    for idx in row_indices:
        # Recupera la serie Y_t dalla riga corrispondente
        Y = dataset.loc[idx, 'Y_t']
        
        # Se necessario, possiamo definire un asse dei tempi
        t = np.arange(len(Y))
        
        # Creiamo la figura
        plt.figure(figsize=(7,4))
        plt.plot(t, Y, color = 'green', linestyle = '--', marker = 'o', markersize = 3.7,label=f'Row {idx}')
        plt.xlabel('Time index')
        plt.ylabel('Amplitude')
        plt.grid(alpha = 0.3)
        
        # Recuperiamo i valori da inserire nel titolo
        subject_id = dataset.loc[idx, 'subject_id']
        trial_id   = dataset.loc[idx, 'trial_id']
        w_noise, w_stim = dataset.loc[idx, '(noise_weight , signal_weight)']
        S_val      = dataset.loc[idx, 'S']
        
        # Creiamo un titolo informativo
        plt.title(
            f'Y(t) - row={idx}, subj={subject_id}, trial={trial_id}, S={S_val}\n'
            f'w_noise={w_noise:.2f}, w_stim={w_stim:.2f}'
        )
        
        plt.legend()
        plt.show()


# DATASET CREATION


In [133]:
# Simulation parameters
n_weights = 10
w_values = np.linspace(0, 1, num=n_weights)
std_X_noise = 2
std_X_ratio = 0.4
std_X_signal = std_X_ratio*std_X_noise
eps = 1e-52 
n_simulations = 50
n_of_Trials = 50
         
# Time parameters 
simLen = 50                                 # simulation length in units of 10 ms
stimWindow = [20, 25]                       # time window of the stimulus
delays = [4,5,6]                            # time delay for Y


# Binning parameters
S_values = [1,2,3,4]
X_bins = 3
Y_bins = 3



dataset = pd.DataFrame(columns=['subject_id', 'weight_id','trial_id', '(noise_weight , signal_weight)', 'S', 'delay', 't', 'X_noise','X_stimulus', 'Y_t'])
w_vector = [(val_i, val_j) for val_i in w_values for val_j in w_values]
t = np.arange(simLen)
t_start = stimWindow[0] # time when stimulus starts

dataset['subject_id']                      = np.repeat(np.arange(n_simulations), n_of_Trials*n_weights*n_weights)
dataset['weight_id']                       = np.tile(np.repeat(np.arange(n_weights*n_weights),n_of_Trials),n_simulations)
dataset['trial_id']                        = np.tile(np.arange(n_of_Trials), n_simulations*n_weights*n_weights)
dataset['(noise_weight , signal_weight)']  = [w_vector[int(idx)] for idx in dataset['weight_id']]
dataset['S']                               = np.random.choice(S_values, n_of_Trials*n_simulations*n_weights*n_weights)
dataset['delay']                           = np.repeat(np.random.choice(delays, n_simulations), n_of_Trials*n_weights*n_weights)
#dataset['dalay']                           = np.random.choice(delays, n_of_Trials*n_simulations*n_weights*n_weights)
dataset['t']                               = t_start + dataset['delay']


X_noise_matrix = np.random.normal(0, std_X_noise, size=(len(dataset), simLen))
dataset['X_noise'] = list(X_noise_matrix)

dataset['X_stimulus'] = dataset['S'].apply(lambda x: 
                                           (np.heaviside(t - stimWindow[0], x) - np.heaviside(t - stimWindow[1], x))*(1 + np.random.normal(0, std_X_signal, simLen)))

dataset['Y_t'] = dataset.apply(lambda row: compute_Y(row, sigma=std_X_signal), axis=1)

In [139]:
dataset['X_noise'].values


array([array([-2.07733565, -1.89705543, -0.32995597, -0.51996295, -1.7161047 ,
               0.08134817, -1.06886999, -4.04558186, -1.02538495,  0.32894019,
              -1.33610148, -1.38117546,  0.05504254, -2.42494029,  1.50662234,
               2.53545862, -0.65601372, -1.45271415, -1.54272174, -1.75962292,
               0.19647901,  1.01952322, -1.15411343, -1.81984865,  1.36455171,
              -3.77613508, -1.99303744, -3.22732426, -1.58604837, -1.49975867,
              -0.50384004, -1.25009032, -0.07640344,  0.46274191,  2.28191524,
              -0.47358917, -1.19615806,  2.00670736,  1.18343807,  2.67104751,
               1.8413405 ,  3.51156423,  1.35180939,  3.63765896, -1.64043334,
              -1.58809365,  1.12113143, -1.65576014, -1.56573346,  0.66244159]),
       array([-1.3938072 ,  4.07003074, -2.72205424,  3.02250082, -3.61185786,
              -1.59087137,  2.46346852,  1.90857267, -2.64181789, -0.08199445,
               0.62896806, -2.68651777, -2.4683022

In [114]:
#minimi = dataset.groupby(['subject_id', 'weight_id']).apply( lambda g: pd.cut(g['X_noise'], 3, retbins=True) )

In [82]:
dataset

,subject_id,weight_id,trial_id,"(noise_weight , signal_weight)",S,delay,t,X_noise,X_stimulus,Y_t
0,0,0,0,"(0.0, 0.0)",1,4,24,"[2.2838642774396805, 1.8703359585441839, -0.68...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.2928949408194908, -0.4500097989040396, -0...."
1,0,0,1,"(0.0, 0.0)",1,4,24,"[0.6485519438259322, -2.784373190581376, -0.57...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0,...","[-0.5821826184761135, 0.11635600561509352, 0.1..."
2,0,0,2,"(0.0, 0.0)",2,4,24,"[-0.5625729266679357, -0.5325237192559673, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.10475963276306098, 1.4499296758847549, -1.0..."
3,0,0,3,"(0.0, 0.0)",3,4,24,"[-1.4358512304626632, -0.879525060830123, -0.1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0,...","[0.4354230532257565, 0.7012411458659344, -1.10..."
4,0,0,4,"(0.0, 0.0)",3,4,24,"[-2.655812814820141, 0.3611731670371928, -1.20...","[0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.06962418780385546, 0.02527916616454136, 0.0..."
...,...,...,...,...,...,...,...,...,...,...
249995,49,99,45,"(1.0, 1.0)",2,4,24,"[0.7823820899395558, -2.2270763464699246, 0.32...","[0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1.1469310848967755, 2.956760088586633, -2.021..."
249996,49,99,46,"(1.0, 1.0)",2,4,24,"[-3.763376114555259, -0.4348844070019119, -0.4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.9948618197194166, -1.520772645831103, 0.75..."
249997,49,99,47,"(1.0, 1.0)",1,4,24,"[1.318013957167265, 0.7361007496849179, -1.891...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-5.049376610035123, -0.5556113140409289, -1.1..."
249998,49,99,48,"(1.0, 1.0)",2,4,24,"[-0.1550780977733922, 0.5431941511479079, 1.46...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.021809245275369787, -1.1010598277324726, 2...."


In [14]:
_, bin_edges = pd.cut(X_noise_matrix[t_start,:], X_bins, retbins=True)
bin_edges



array([-4.59049197, -1.42904788,  1.72294024,  4.87492836])

In [530]:
np.digitize(dataset['X_noise'].iloc[dataset['S']][:][0], bins=bin_edges, right=True)

KeyError: 0